# 30. 异常对象（Exception Objects）

异常也是对象：有类型、有 args、有 traceback；还可能带 __cause__/__context__。掌握这些有助于日志与排障。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 28-29 节：异常基础与链


## 知识点地图

- 1. 异常对象：type(e)/e.args/str(e)
- 2. __cause__：raise ... from ... 指定的原因
- 3. __context__：异常处理中又发生异常（隐式链）
- 4. traceback：format_exc 获取堆栈字符串


## 自检清单（学完打勾）

- [ ] 会读取异常类型与 args
- [ ] 理解 __cause__ 与 __context__ 的区别
- [ ] 会用 traceback.format_exc 获取堆栈字符串
- [ ] 了解在生产环境记录异常的注意点（敏感信息）


## 知识点 1：异常对象：type(e)/e.args/str(e)

捕获异常后可以查看其类型、参数与字符串表示。


In [ ]:
try:
    int('x')
except Exception as e:
    print(type(e))
    print('args:', e.args)
    print('str :', str(e))


## 知识点 2：__cause__：raise ... from ... 指定的原因

__cause__ 来自显式 raise from；用于保留“原始异常”。


In [ ]:
try:
    try:
        int('x')
    except ValueError as e:
        raise RuntimeError('wrap') from e
except Exception as e:
    print(type(e).__name__)
    print('cause:', type(e.__cause__).__name__)


## 知识点 3：__context__：异常处理中又发生异常（隐式链）

如果在处理异常时又发生新异常，Python 会把原异常放到 __context__。


In [ ]:
try:
    try:
        1 / 0
    except ZeroDivisionError:
        int('x')
except Exception as e:
    print(type(e).__name__)
    print('context:', type(e.__context__).__name__)


## 知识点 4：traceback：format_exc 获取堆栈字符串

traceback.format_exc() 常用于日志；注意生产环境避免泄露敏感信息。


In [ ]:
import traceback
try:
    1 / 0
except Exception:
    tb = traceback.format_exc()
    print(tb.splitlines()[-1])


## 常见坑

- 不要把完整 traceback 直接返回给用户（可能泄露路径/配置/数据）
- 捕获后重新抛出要用 raise（不要 raise e，会改变 traceback）


## 综合小案例：把异常写入日志文件（error.log）

捕获异常后把 traceback 写入 _nb_artifacts/error.log，并打印最后一行。


In [ ]:
from pathlib import Path
ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())
import traceback

log_path = ART / 'error.log'
try:
    1 / 0
except Exception:
    tb = traceback.format_exc()
    log_path.write_text(tb, encoding='utf-8')
    print(tb.splitlines()[-1])
    print('written to', log_path)


## 自测题（不写代码也能回答）

- __cause__ 与 __context__ 分别来自什么情况？
- 为什么不建议把 traceback 原样展示给用户？
- traceback.format_exc 输出什么？


## 练习题（建议写代码）

- 写一个函数 safe_call(fn, *args, **kwargs)：捕获异常后返回 (None, error_str)。
- 用 raise from 包装异常并打印 e.__cause__。
